# Get Test Set

A ADS with all w values in all scenarios, expect to have 5*6*20=600 items

Save in a dict. 1st layer key is scenario name, 2nd layer key is w, value is params

In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
from pathlib import Path
from IPython.display import display
import pickle
# from env_utils import read_pso_search_files, detect_convergence
%matplotlib inline

from data_utils import get_unique_search_df

In [2]:
def get_final_test_set(exp_path):
    # Analyze species, return a list of the particles's best value
    with open(exp_path / "species.pkl", "rb") as f:
        species = pickle.load(f)
    final_test_set = []
    for s in species:
        for p in s:
            if p.best is not None:
                final_test_set.append(p.best)
    return final_test_set

def get_final_test_set_unique_20(exp_path):
    # Analyze species, return a list of the particles's best value
    search_df_unique = get_unique_search_df(exp_path)
    search_df_last20 = search_df_unique.tail(20)
    final_test_set = []
    for index, row in search_df_last20.iterrows():
        final_test_set.append(row[7:].to_list())
    return final_test_set

In [3]:
ADS_name = "apollo"

test_set_dict = {}

# Set the base path of the output directory
base_path = Path("../../output-final")

# Traverse the output directory
for scenario_name in os.listdir(base_path):
    test_set_dict[scenario_name] = {}
    # Check if it is a directory
    scenario_path = os.path.join(base_path, scenario_name)
    if os.path.isdir(scenario_path):
        # Traverse the agent directory in each scenario
        for agent_name in os.listdir(scenario_path):
            if agent_name != ADS_name:
                continue
            agent_path = os.path.join(scenario_path, agent_name)
            if os.path.isdir(agent_path):
                for experiment_name in os.listdir(agent_path):

                    # Only need results from full test
                    if "20x50" not in experiment_name:
                        continue

                    w = float(experiment_name.split('_')[1])

                    test_set_dict[scenario_name][w] = []

                    exp_path = os.path.join(agent_path, experiment_name)

                    # get test set of this experiment
                    final_test_set = get_final_test_set_unique_20(Path(exp_path))

                    test_set_dict[scenario_name][w] = final_test_set            
                    

In [4]:
# Show the second layer structure of the dict
for k, v in test_set_dict.items():
    print(k)
    for k2, v2 in v.items():
        print(k2, len(v2))
    print()

nonsignalized_junction_left_turn
0.5 20
1.0 20
0.7 20
0.0 20
0.3 20

front_cut_in_with_two_npc
0.0 20
0.7 20
1.0 20
0.3 20
0.5 20

opposite_vehicle_taking_priority
1.0 20
0.0 20
0.5 20
0.7 20
0.3 20

front_cut_in_with_one_npc
0.5 20
0.0 20
1.0 20
0.3 20
0.7 20

front_brake
0.7 20
0.0 20
1.0 20
0.5 20
0.3 20

nonsignalized_junction_right_turn
0.3 20
1.0 20
0.5 20
0.7 20
0.0 20



In [5]:
# save test_set_dict as pickle
with open(f"test_set_{ADS_name}.pkl", "wb") as f:
    pickle.dump(test_set_dict, f)